In [1]:
import scanpy as sc
import numpy as np
import pandas as pd
from flecs.utils import get_project_root
from flecs.data.utils import load_interaction_data
import os

## Load original data

In [34]:
adata = sc.read_h5ad(os.path.join(get_project_root(), 
                                     "datasets/PerturbSeq/"
                                     "K562_gwps_raw_singlecell_01.h5ad"), 
                     backed="r")

In [35]:
adata

AnnData object with n_obs × n_vars = 1989578 × 8248 backed at '/Users/paul/PycharmProjects/FLeCS/datasets/PerturbSeq/K562_gwps_raw_singlecell_01.h5ad'
    obs: 'gem_group', 'gene', 'gene_id', 'transcript', 'gene_transcript', 'sgID_AB', 'mitopercent', 'UMI_count', 'z_gemgroup_UMI', 'core_scale_factor', 'core_adjusted_UMI_count'
    var: 'gene_name', 'chr', 'start', 'end', 'class', 'strand', 'length', 'in_matrix', 'mean', 'std', 'cv', 'fano'

## Restrict to a few perturbations

In [36]:
# We restrict ourselves to KO that targetted two promoters
adata_p1p2 = adata[adata.obs["transcript"].apply(lambda t: t=="P1P2" or t=="non-targeting")]

In [37]:
# We focus on genes for which at least 1000 cells have been KO
v_counts = adata_p1p2.obs["gene"].value_counts()
important_KO_genes = list(v_counts[v_counts>600].index)
important_KO_genes.remove('non-targeting')

In [38]:
print(len(important_KO_genes))

34


In [39]:
# We keep just 10% of observational data
indices_to_keep = (adata.obs["gene"].apply(lambda g: g=='non-targeting') \
                   & (np.random.uniform(size=len(adata)) > 0.9))

for gene in important_KO_genes:
    indices_to_keep = indices_to_keep | adata.obs["gene"].apply(lambda g: g==gene)    

In [40]:
adata_subset = adata[indices_to_keep].to_memory()

## Rename variables

In [41]:
adata_subset.var["gene_name"].value_counts()

HSPA14    2
TBCE      2
PUS7L     1
PUM3      1
PURA      1
         ..
GNA13     1
GNA12     1
GNA11     1
GMPS      1
ZZEF1     1
Name: gene_name, Length: 8246, dtype: int64

In [42]:
# Let us remove duplicated genes from variables
HSPA14_index_0 = adata_subset.var[adata_subset.var["gene_name"] == "HSPA14"].index[0]
TBCE_index_0 = adata_subset.var[adata_subset.var["gene_name"] == "TBCE"].index[0]

In [43]:
# Remove duplicated gene names
var_names = list(adata_subset.var_names)
var_names.remove(HSPA14_index_0)
var_names.remove(TBCE_index_0)
adata_subset = adata_subset[:, var_names]

In [44]:
adata_subset

View of AnnData object with n_obs × n_vars = 36013 × 8246
    obs: 'gem_group', 'gene', 'gene_id', 'transcript', 'gene_transcript', 'sgID_AB', 'mitopercent', 'UMI_count', 'z_gemgroup_UMI', 'core_scale_factor', 'core_adjusted_UMI_count'
    var: 'gene_name', 'chr', 'start', 'end', 'class', 'strand', 'length', 'in_matrix', 'mean', 'std', 'cv', 'fano'

In [45]:
adata_subset.var.set_index(adata_subset.var["gene_name"].astype('object'), inplace=True)

## Load K562 GRN

In [46]:
grn = load_interaction_data(interaction_type="fantom5", realnet_tissue_type_file="15_myeloid_leukemia.txt.gz")

In [47]:
grn

InteractionData. 14969 nodes and 2238288 edges.
2 different types of nodes: ['TF_gene', 'gene'].
2 different types of edges: [('TF_gene', '', 'gene'), ('TF_gene', '', 'TF_gene')].

In [48]:
node_names = [v['name'] for v in grn.node_data().values()]

In [49]:
len(node_names)

14969

In [50]:
len(np.unique(node_names))

14969

## Map GRN gene names to the variable names in the adata

In [51]:
intersection_names = list(set(adata_subset.var_names).intersection(node_names))

In [52]:
len(intersection_names)

7361

In [53]:
# Take the subgraph restricted to nodes in the intersection
kept_nodes = [k for k, v in grn.node_data().items() if v['name'] in intersection_names]

In [54]:
sub_grn = grn.to_digraph().subgraph(kept_nodes)

In [55]:
edges = list(sub_grn.edges())

## Subset adata and build GRN adjacency matrix

In [56]:
adata_subset = adata_subset[:, intersection_names]

In [57]:
adata_subset

View of AnnData object with n_obs × n_vars = 36013 × 7361
    obs: 'gem_group', 'gene', 'gene_id', 'transcript', 'gene_transcript', 'sgID_AB', 'mitopercent', 'UMI_count', 'z_gemgroup_UMI', 'core_scale_factor', 'core_adjusted_UMI_count'
    var: 'gene_name', 'chr', 'start', 'end', 'class', 'strand', 'length', 'in_matrix', 'mean', 'std', 'cv', 'fano'

In [58]:
grn_adj_mat = np.zeros((adata_subset.shape[1], adata_subset.shape[1]))

In [59]:
names_to_new_index = {n: i for i, n in enumerate(intersection_names)}

In [60]:
old_index_to_name = {k: v["name"] for k, v in grn.node_data().items()}

In [61]:
named_edges = [(old_index_to_name[x[0]], old_index_to_name[x[1]]) for x in edges]
reindexed_edges = [(names_to_new_index[x[0]], names_to_new_index[x[1]]) for x in named_edges]

In [62]:
len(reindexed_edges)

456670

In [63]:
# Build adjacency matrix of the GRN, in the order consistent with adata_subset
for edge in reindexed_edges:
    grn_adj_mat[edge[0], edge[1]] = 1

In [64]:
adata_subset.varp["grn_adj_mat"] = grn_adj_mat

## Save

In [65]:
adata_subset.write_h5ad(os.path.join(get_project_root(), 
                                     "datasets/PerturbSeq/processed/"
                                     "K562_gwps_normalized_singlecell_with_grn.h5ad"))